<a href="https://colab.research.google.com/github/JesseJames50/projeto_llm/blob/main/Agentes_com_m%C3%BAltiplas_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="color: red; font-weight: bold;">ATENÇÃO ERRATA</h1>


Na Live eu disse para colocar o comando "allow_delegation=False" nas tarefas.
</br>
Na verdade deve ser nos agentes.
</br>
O código já está corrigido.
</br>
Pedimos desculpas pelo engano.

# AGENTES COM MÚLTIPLAS LLM'S

In [2]:
!pip install --q langchain
!pip install --q openai
!pip install --q crewai
!pip install --q 'crewai[tools]'
!pip install --q python-dotenv
!pip install --q textwrap3

<img src="https://raw.githubusercontent.com/sandeco/crewai/main/images/2.png" width="80%"
/>

In [3]:
import os
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from crewai_tools.tools.scrape_website_tool.scrape_website_tool import ScrapeWebsiteTool
from dotenv import load_dotenv
from datetime import datetime
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from textwrap import dedent

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [4]:
# Carregar as variáveis de ambiente do arquivo .env
load_dotenv()

# Configuração das chaves de API (substitua com suas chaves reais)
openai_api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

# Configuração das chaves de API (substitua com suas chaves reais)
openai_api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
drant_key = os.getenv("QDRANT_API_KEY")

print(f"OPENAI_API_KEY: {openai_api_key}")
print(f"GROQ_API_KEY: {groq_api_key}")
print(drant_key)

OPENAI_API_KEY: sk-dUBI5uSLf65tg34cRXWuAShvrGVOw_hXKl_uOTjolRT3BlbkFJyP8FR1a1sFrtVOU9EeFGsBMj7OTSyFa_yiEOASqtsA
GROQ_API_KEY: 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
pQz6rFT_sqlhQ0RG4yWX7EeN8ZmReWRBQVsG3vMWRJN0Pl2S0esDrg


# NOSSAS MENTES

In [5]:
gpt3_llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=openai_api_key)

In [6]:
gpt4o_llm = ChatOpenAI(model="gpt-4o", api_key=openai_api_key)

In [7]:
llama3_70b = ChatGroq(model="llama3-70b-8192", api_key=groq_api_key)

#RASPANDO A WEB

In [8]:
ainew = ScrapeWebsiteTool(
    website_url="https://www.artificialintelligence-news.com/"
)

In [9]:
forbes = ScrapeWebsiteTool(
    website_url="https://www.forbes.com/ai/"
)

#CRIANDO OS AGENTES

In [10]:
planejador = Agent(
    role="Planejador de postagem",
    goal="Planejar conteúdo envolvente para instagram sobre {topic}",
    backstory="Você está trabalhando no planejamento de {n} posts para o instagram "
              "sobre o tema: {topic}. "
              "Você coleta informações que ajudam o "
              "público se informar sobre {topic}. "
              "Seu trabalho é a base para que "
              "o Pesquisador de Conteúdo procure na web sobre {topic}.",
    verbose=True,
    llm=gpt3_llm,
    allow_delegation=False

)

In [11]:
pesquisador = Agent(
    role='Pesquisador',
    goal='Pesquisar tendências para postagens sobre {topic} na área '
         'de tecnologia com base no planejamento do Planejador. '
         'Seu trabalho é a base para que '
         'o escritor possa escrever {n} posts sobre {topic}',
    verbose=True,
    backstory="Você é um pesquisador experiente, sempre em busca das últimas tendências e informações relevantes sobre {topic}.",
    llm=gpt3_llm,
    tools=[ainew, forbes],
    allow_delegation=False
)


In [12]:
escritor = Agent(
    role='Escritor',
    goal='Escrever {n} postagens cativantes em português do Brasil para o Instagram sobre {topic} com no mínimo 250 palavras e no máximo 350 palavras.'
         'Seu trabalho é a base para que o fotografo possa escrever prompts de imagens para os {n} posts',
    backstory="Você é um escritor criativo, capaz de transformar informações em conteúdo atraente para postagens no Instagram.",
    llm=gpt4o_llm,
    verbose=True,
    allow_delegation=False
)

In [13]:
fotografo = Agent(
    role='Fotógrafo',
    goal='Escrever prompts de imagens para as {n} postagens para gerar imagens cativantes para o Instagram sobre {topic}.',
    backstory=dedent("""Você é um fotógrafo criativo,
                        capaz de transformar informações em imagens e escrever prompts
                        de imagens atraentes para postagens no Instagram."""),
    verbose=True,
    llm=llama3_70b,
    max_rpm=1,
    allow_delegation=False
)

In [14]:
# Agentes
gerente = Agent(
    role='Gerente de postagens',
    goal=dedent("""Supervisione o trabalho de uma equipe de postagens no Instagram. Você é bem crítico em relação
            ao que vai ser postado no Instagram da empresa de notícias na área da tecnologia.
            Você delegará tarefas à sua equipe e fará perguntas esclarecedoras
            para revisar e aprovar as {n} posts sobre {topic} que foram solicitadas pela direção da empresa."""),
    verbose=True,
    backstory=dedent("""Você é um gerente experiente, sempre em busca das últimas tendências e informações relevantes.
                 Você está trabalhando com uma nova demanda e faz com que sua equipe realize o trabalho da
                 melhor forma possível."""),
    llm=gpt3_llm,
)

#TAREFAS

In [15]:
plano_task = Task(
    description=(
        "1. Priorize as últimas tendências, principais 'players', "
            "e notícias relevantes sobre {topic}.\n"
        "2. Identifique o público-alvo, considerando "
            "seus interesses e pontos de dor.\n"
        "3. Desenvolva um plano de conteúdo detalhado, incluindo "
            "uma introdução, pontos principais e um chamado à ação.\n"
        "4. Inclua palavras-chave de SEO e dados ou fontes relevantes."
    ),
    expected_output="Um documento de plano de conteúdo para {n} posts sobre {topic} "
        "com um esboço, análise do público, "
        "palavras-chave de SEO e recursos.",
    agent=planejador,
    verbose=2
)

In [16]:
pesquisa_task = Task(
    description="Pesquise as últimas tendências sobre {topic}.",
    expected_output="Um relatório detalhado sobre as tendências mais recentes sobre {topic} na área de tecnologia.",
    agent=pesquisador,
    verbose=2
)

In [18]:
escrita_task = Task(
    description=dedent("""Escreva {n} postagens envolventes em português do Brasil
                        com base nas tendências pesquisadas sobre {topic} com no mínimo 250 palavras e no máximo 350 palavras cada.
                        Cada post deve ser formatado como:
                        \n\nPOST:\ntexto do post em português do brasil
                        \n\nPROMPT:\nPrompt da imagem desse post\n\n"""),
    expected_output="{n} postagens de Instagram bem escritas, atraentes e em português do Brasil, formatadas conforme especificado para o tópico {topic}.",
    agent=escritor,
    verbose=2
)

In [17]:
criacao_imagem_task = Task(
    description="Crie {n} prompts para criar uma imagem atraente para acompanhar a postagem no Instagram sobre {topic}.",
    expected_output="{n} prompts de alta qualidade adequados para o Instagram based in {topic}.",
    agent=fotografo,
    verbose=2
)

In [19]:
revisao_task = Task(
    description=dedent("""Revise as {n} escritas e prompts de imagens
                          para as {n} postagens do cliente e garanta
                          que as informações de cada postagem estejam organizadas, sem erros e cativantes
                          em português do Brasil sobre {topic}.
                          Certifique-se de que cada post está formatado como:
                          \n\nPOST:\ntexto do post em português do brasil
                          \n\nPROMPT:\nPrompt da imagem desse post\n\n"""),
    expected_output="{n} textos e prompts de imagens organizados por post, revisados e prontos para serem publicados em português do Brasil, formatados conforme especificado.",
    agent=gerente,
    verbose=2
)

#CRIANDO A TRIPULAÇÃO

In [20]:
crew = Crew(
    agents=[planejador, pesquisador, escritor, fotografo, gerente],
    tasks=[plano_task, pesquisa_task, escrita_task, criacao_imagem_task, revisao_task],
    process=Process.sequential,
    verbose=2,
    memory=True)

<img src="https://raw.githubusercontent.com/sandeco/crewai/main/images/15.png" width="80%"
/>

In [21]:
result = crew.kickoff(inputs={
      'topic':'Inteligência Artificial e Agentes Inteligentes',
      'n':1})

 [2024-08-11 20:26:18][DEBUG]: == Working Agent: Planejador de postagem
 [2024-08-11 20:26:18][INFO]: == Starting Task: 1. Priorize as últimas tendências, principais 'players', e notícias relevantes sobre Inteligência Artificial e Agentes Inteligentes.
2. Identifique o público-alvo, considerando seus interesses e pontos de dor.
3. Desenvolva um plano de conteúdo detalhado, incluindo uma introdução, pontos principais e um chamado à ação.
4. Inclua palavras-chave de SEO e dados ou fontes relevantes.


> Entering new CrewAgentExecutor chain...
I now have a clear understanding of how to create engaging content for Instagram on the topic of Artificial Intelligence and Intelligent Agents.

Final Answer:
Title: Unlocking the Power of Artificial Intelligence and Intelligent Agents

Introduction:
Artificial Intelligence (AI) and Intelligent Agents are revolutionizing the way we interact with technology. From virtual assistants to self-driving cars, AI is reshaping the world around us. In this p

Thought: I should use the available tool to gather information from relevant websites and provide a detailed report on the latest trends in Artificial Intelligence and Intelligent Agents.

Action: Read website content
Action Input: {}
 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Final Answer: 
AI News - Artificial Intelligence News
News
Categories
Applications
Chatbots
Face Recognition
Virtual Assistants
Voice Recognition
Companies
Amazon
Apple
Google
Meta (Facebook)
Microsoft
NVIDIA
Deep & Reinforcement Learning
Enterprise
Ethics & Society
Industries
Energy
Entertainment & Retail
Gaming
Healthcare
Manufacturing
Legislation & Government
Machine Learning
Privacy
Research
Robotics
Security
Surveillance
Events
Webinars & Resources
Work With Us
Publish an article or press release
Advertise on AI News
Contact Us
Subscribe / Login
More TechForge News
The Block
Cloud Tech News
Developer News
Edge Computing News
IoT News
Marketing 

In [23]:
current_date = datetime.now().strftime("%Y-%m-%d")
filename = f"posts-{current_date}.txt"              #"post-2024-06-11.txt"
with open(filename, 'w', encoding='utf-8') as file:
    file.write(str(result))

In [24]:
#Teste da api openai
from openai import OpenAI
client = OpenAI(api_key= openai_api_key)
response  = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "O que é inteligência artificial?"}
    ]
)
# Acesse a resposta diretamente do objeto retornado
analysis = response.choices[0].message.content.strip()
print(analysis)

Inteligência Artificial (IA) é um ramo da ciência da computação que busca criar sistemas capazes de realizar tarefas que, normalmente, requerem inteligência humana. Isso inclui habilidades como raciocínio, aprendizado, percepção, compreensão da linguagem e tomada de decisões. A IA pode ser dividida em diversas categorias, como:

1. **IA Fraca (ou Narrow AI)**: Refere-se a sistemas projetados para realizar uma tarefa específica, como assistentes virtuais (ex: Siri, Alexa), sistemas de recomendação, ou programas de reconhecimento de imagem. Esses sistemas não possuem consciência ou entendimento além da tarefa para a qual foram programados.

2. **IA Forte (ou General AI)**: É um conceito teórico que se refere a sistemas que teriam a capacidade de entender, aprender e aplicar inteligência em uma ampla gama de tarefas, similar ao ser humano. Até o momento, essa forma de IA ainda não foi desenvolvida.

3. **Aprendizado de Máquina (Machine Learning)**: Uma subárea da IA que envolve o desenvol